In [1]:
!pip install transformers datasets evaluate librosa torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `imlla` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

In [3]:
from datasets import load_dataset


dataset = load_dataset("FBK-MT/Speech-MASSIVE-test", 'pl-PL', split='test', trust_remote_code=True)
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

test-00000-of-00003.parquet:   0%|          | 0.00/335M [00:00<?, ?B/s]

test-00001-of-00003.parquet:   0%|          | 0.00/337M [00:00<?, ?B/s]

test-00002-of-00003.parquet:   0%|          | 0.00/335M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2974 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'locale', 'partition', 'scenario', 'scenario_str', 'intent_idx', 'intent_str', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments', 'tokens', 'labels', 'audio', 'path', 'is_transcript_reported', 'is_validated', 'speaker_id', 'speaker_sex', 'speaker_age', 'speaker_ethnicity_simple', 'speaker_country_of_birth', 'speaker_country_of_residence', 'speaker_nationality', 'speaker_first_language'],
    num_rows: 2974
})


In [4]:
dataset = dataset.select_columns(['id', 'scenario_str', 'intent_str', 'utt', 'tokens', 'labels', 'audio', 'path'])
dataset[0]

{'id': '11861',
 'scenario_str': 'recommendation',
 'intent_str': 'recommendation_movies',
 'utt': 'jaki film jest teraz najwyżej oceniany',
 'tokens': ['jaki', 'film', 'jest', 'teraz', 'najwyżej', 'oceniany'],
 'labels': ['Other', 'Other', 'Other', 'Other', 'Other', 'Other'],
 'audio': {'path': 'dab7ab8b100ece47b0a1e822a6beac5b.wav',
  'array': array([-6.98491931e-10,  8.14907253e-10, -8.14907253e-10, ...,
         -7.34902860e-04, -6.60274061e-04, -4.50702908e-04]),
  'sampling_rate': 16000},
 'path': 'test/dab7ab8b100ece47b0a1e822a6beac5b.wav'}

In [14]:
import librosa
import os
from transformers import WhisperProcessor


processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")

def preprocess_function(batch):

    audio = batch["audio"]
    input_features = processor(
        audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt"
    ).input_features

    batch["input_features"] = input_features
    batch["labels"] = processor.tokenizer(batch["utt"]).input_ids

    return batch

In [15]:
processed_dataset = dataset.map(preprocess_function)

Map:   0%|          | 0/2974 [00:00<?, ? examples/s]

In [16]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [26]:
import torch

def transcribe(batch):
    input_features = batch["input_features"]
    input_features = torch.tensor(input_features)

    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription


results = processed_dataset.select(range(5)).map(lambda x: {"transcription": transcribe(x)})

for result in results:
    print(f"Original Text: {result['utt']}")
    print(f"Transcription: {result['transcription']}")
    print("-" * 50)

Original Text: jaki film jest teraz najwyżej oceniany
Transcription:  Jak i film jest teraz najwyżlioteniony.
--------------------------------------------------
Original Text: czy są jakieś informacje o prezydenturze
Transcription:  Czy są jakieś informacje o prezetyntu, że
--------------------------------------------------
Original Text: usuń następne wydarzenie dzisiaj
Transcription:  Musun na stampę wydarzenie dzisiaj.
--------------------------------------------------
Original Text: nowe aktualności
Transcription:  Nowe aktualności.
--------------------------------------------------
Original Text: czy mogę zamówić dostawę z tej restauracji
Transcription:  Czy mogę zamówić dostawę ustaj restauracji?
--------------------------------------------------
